In [56]:
import numpy as np
import scipy
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from scipy.stats import *
import math


from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from scipy.stats import *
from sklearn.metrics import*
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage,dendrogram 

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector, RFE
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold, GridSearchCV

Since this is a binary classification problem(supervised), we will consider the following models: LogisticRegression, Random Forest Classifiers, KNN classifier and Support Vector Multiplier. We will use cross validation(mean accuracy) to evaluate the model and use gridsearchCV, RandomizedSearchCV to find the best tuned hyperparameters for each model. The first step is to filter the data such that we can eliminate correlation and pick the most useful features by check importance.


In [3]:
# import the data sets
df = pd.read_csv('/Users/qinshaofeng/Desktop/titanic/train.csv')

In [67]:
# transform sex to 0/1
le = LabelEncoder()
df["Sex"] = le.fit_transform(df["Sex"])
df["Age"] = df["Age"].fillna((df["Age"].mean()))
print(df.columns)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [11]:
# discard string names, passenger and ticket numbers, embarked since they are not quite relevant with our prediction
# I also ignore the Cabin since there are too many NaN in it
X = df[['Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare']]
y = df['Survived']
# note X, y are our training set already, no need to split

In [30]:
# feature selection: forward and backward; 
#check for knn
knn = KNeighborsClassifier()
sfs = SequentialFeatureSelector(knn, direction = 'forward', n_features_to_select=4)
sfs.fit(X, y)

sfs.get_support()

array([ True,  True, False, False,  True,  True])

In [31]:
# feature selection: forward and backward; 
#check for knn
knn = KNeighborsClassifier()
sfs = SequentialFeatureSelector(knn, direction = 'backward', n_features_to_select=4)
sfs.fit(X, y)
sfs.get_support()

array([ True,  True,  True, False,  True, False])

In [32]:
# try for random forest

rdf = RandomForestClassifier()
sfs = SequentialFeatureSelector(rdf, direction = 'backward', n_features_to_select=4)
sfs.fit(X, y)
sfs.get_support()


array([ True,  True,  True, False, False,  True])

In [33]:
rdf = RandomForestClassifier()
sfs = SequentialFeatureSelector(rdf, direction = 'forward', n_features_to_select=4)
sfs.fit(X, y)
sfs.get_support()

array([ True,  True,  True, False, False,  True])

In [34]:
# feature selection: forward and backward; 
#check for knn
lgr = LogisticRegression()
sfs = SequentialFeatureSelector(lgr, direction = 'backward',n_features_to_select=4)
sfs.fit(X, y)
sfs.get_support()

array([ True,  True, False,  True,  True, False])

In [36]:
# feature selection: forward and backward; 
#check for knn
lgr = LogisticRegression()
sfs = SequentialFeatureSelector(lgr, direction = 'forward',n_features_to_select=4)
sfs.fit(X, y)
sfs.get_support()

array([ True,  True, False,  True,  True, False])

In [72]:
# based on the vote above, we select 
X = df[['Pclass', 'Sex', 'Age','Parch', 'Fare']]
Y = df['Survived']
X_test = pd.read_csv('/Users/qinshaofeng/Desktop/titanic/test.csv')
ID = X_test['PassengerId']

In [73]:
# load the testing set
X_test = X_test[['Pclass', 'Sex', 'Age','Parch', 'Fare']]
le = LabelEncoder()
X_test["Sex"] = le.fit_transform(X_test["Sex"])
X_test["Age"] = X_test["Age"].fillna((X_test["Age"].mean()))
X_test["Fare"] = X_test["Fare"].fillna((X_test["Fare"].mean()))


In [38]:
# use different models and find the best tunned parameters
# Random forest
leaf_nodes = np.arange(50, 200, 20)
n_trees = np.arange(100, 300,20)
param_grid = {'n_estimators':n_trees, 'max_leaf_nodes':leaf_nodes}
clf = RandomForestClassifier(random_state = 20)
grid_clf = GridSearchCV(clf, param_grid, cv=10, scoring = 'accuracy')
grid_clf.fit(X, y)
grid_clf.best_estimator_ 

RandomForestClassifier(max_leaf_nodes=50, n_estimators=120, random_state=20)

In [76]:
X = df[['Pclass', 'Sex', 'Age','Parch', 'Fare']]
Y = df['Survived']
rdf = RandomForestClassifier(max_leaf_nodes=50, n_estimators=120, random_state=20)
rdf.fit(X,y)
rdf_prediction = rdf.predict(X_test)
rdf_result = pd.DataFrame({'PassengerId': ID, 'Survived': rdf_prediction})
rdf_result.to_csv('/Users/qinshaofeng/Desktop/titanic/rdf_report.csv')
#print("with tuned random forest classifer\n prediction is: {}".format(rdf_prediction))
print(rdf_result)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [52]:
neighbors = np.arange(0,20)
param_grid = {'n_neighbors': neighbors}
clf = KNeighborsClassifier()
grid_clf = GridSearchCV(clf, param_grid, cv=10, scoring = 'accuracy')
grid_clf.fit(X,y)
grid_clf.best_estimator_ 

/Users/qinshaofeng/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/qinshaofeng/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/qinshaofeng/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py", line 179, in fit
    return self._fit(X, y)
  File "/Users/qinshaofeng/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_base.py", line 514, in _fit
    raise ValueError(
ValueError: Expected n_neighbors > 0. Got 0

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/qinshaofeng/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit f

KNeighborsClassifier(n_neighbors=7)

In [77]:
# best neighbors = 7
knn= KNeighborsClassifier(n_neighbors = 7)
knn.fit(X,y)
knn_prediction = knn.predict(X_test)
knn_result = pd.DataFrame({'PassengerId': ID, 'Survived': knn_prediction})
knn_result.to_csv('/Users/qinshaofeng/Desktop/titanic/knn_report.csv')
print("with KNN classifer\n prediction is: {}".format(knn_prediction))

with KNN classifer
 prediction is: [0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0
 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0 1 0
 1 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0
 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1
 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 0
 0 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 0
 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0
 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 1 1 1 1 0 0 1 0 0 1]


In [79]:
lgr = LogisticRegression()
lgr.fit(X,y)
lgr_prediction = lgr.predict(X_test)
print("with logistic regression classifer\n prediction is: {}".format(lgr_prediction))
lgr_result = pd.DataFrame({'PassengerId': ID, 'Survived': lgr_prediction})
lgr_result.to_csv('/Users/qinshaofeng/Desktop/titanic/lgr_report.csv')

with logistic regression classifer
 prediction is: [0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [65]:
degree = [3,4,5,6,7,8]
param_grid = {'degree':degree}
grid_clf = GridSearchCV(clf, param_grid, cv=10, scoring = 'accuracy')
grid_clf.fit(X,y)
grid_clf.best_estimator_ 

SVC()

In [80]:
# tuned SVC is the default with degree of 3, kernel is using 'rbf'
svm = SVC()
svm.fit(X,y)
svm_prediction = svm.predict(X_test)
svm_result = pd.DataFrame({'PassengerId': ID, 'Survived': svm_prediction})
svm_result.to_csv('/Users/qinshaofeng/Desktop/titanic/svm_report.csv')
print("with SVM classifer\n prediction is: {}".format(svm_prediction))

with SVM classifer
 prediction is: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 0 0 0 1 0 0 1 0 0 0]
